### API KEY Load

In [1]:
from dotenv import load_dotenv
import os

load_dotenv()
api_key = os.getenv("OPENAI_API_KEY")

### Data Preprocess

In [3]:
import os
import pandas as pd

JD_csv = pd.read_csv('./data/USA_jobs_total.csv')

# num_urgent_words - 이건 무슨 칼럼인지 모르겠움..
# column drop - interval, min_amount, max_amount, currency, num_urgent_words, benefits, emails
JD_csv.drop(['interval', 'min_amount', 'max_amount', 'currency', 'num_urgent_words', 'benefits', 'emails'], axis=1, inplace=True)

# description null인 행만 지우기
JD_csv.dropna(subset=['description', 'is_remote'], inplace=True)

# is_remote null 값 처리 - description을 llm에 전달한 후 job_type 넣기
JD_csv['job_type'] = JD_csv['job_type'].fillna('fulltime')
JD_csv['index'] = [f"job_{i}" for i in range(len(JD_csv))]


JD_csv.info()

<class 'pandas.core.frame.DataFrame'>
Index: 20 entries, 0 to 39
Data columns (total 11 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   job_url      20 non-null     object
 1   site         20 non-null     object
 2   title        20 non-null     object
 3   company      20 non-null     object
 4   company_url  20 non-null     object
 5   location     20 non-null     object
 6   job_type     20 non-null     object
 7   date_posted  20 non-null     object
 8   is_remote    20 non-null     object
 9   description  20 non-null     object
 10  index        20 non-null     object
dtypes: object(11)
memory usage: 1.9+ KB


### null값 채우기 - LLM이용하여 판단하기

In [ ]:
# job_type_null= JD_csv[JD_csv['job_type'].isnull()]['description'].to_list()
# job_type_null

["Riverbed. Empower the Experience:\n\nRiverbed, the leader in AI observability, helps organizations optimize their\nuser's experiences by leveraging AI automation for the prevention,\nidentification, and resolution of IT issues. With over 20 years of experience\nin data collection and AI and machine learning, Riverbed's open and **AI-\npowered observability platform** and solutions optimize digital experiences\nand greatly improves IT efficiency. Riverbed also offers industry-leading\n**Acceleration solutions** that provide fast, agile, secure acceleration of\nany app, over any network, to users anywhere. Together with our thousands of\nmarket-leading customers globally - including 95% of the FORTUNE 100 - we are\nempowering next-generation digital experiences.\n\nPosition:\n\n**Title:** Solutions Engineer\n\n **Location:** Remote/Home Office - greater Chicago, IL or Michigan.\n\n **Technology Specialization:** Unified Observability\n\n  \n\nWe are seeking a Solutions Engineer for imm

In [ ]:
# from langchain_openai import ChatOpenAI
# from langchain_core.output_parsers import StrOutputParser
# from langchain_core.runnables import RunnablePassthrough
# from langchain_core.prompts import PromptTemplate

# prompt = PromptTemplate.from_template('''
# {context}

# Can you tell what type of job this is by reading this article? What type do you think it is: contract, full-time, intern, or part-time? 
# Answer with only one word out of the four. And if you don't know, just say you don't know.
# ''')

# llm = ChatOpenAI(model='gpt-3.5-turbo')

# chain = (
#     {"context": RunnablePassthrough()}
#     | prompt
#     | llm
#     | StrOutputParser()
# )

# results = []
# for context in job_type_null:
#     response = chain.invoke(context)
#     results.append(response)
# print(results)

['Full-time', 'Full-time', 'Full-time', 'Full-time', 'Full-time']


In [4]:
JD_csv.head(5)

,job_url,site,title,company,company_url,location,job_type,date_posted,is_remote,description,index
0,https://www.indeed.com/viewjob?jk=301b28f66c77...,indeed,Engineer,Realign LLC,https://www.indeed.com/cmp/Realign,"Minneapolis, MN, USA",contract,2024-11-12,False,##### **Job Type: Contract**\n\n##### **Job C...,job_0
1,https://www.indeed.com/viewjob?jk=e80b0e14767a...,indeed,Product Manager,Wheel,https://www.indeed.com/cmp/Wheel-5,"Remote, USA",fulltime,2024-11-11,True,"Location\n\nRemote, USA\n\nType\n\nFull time\n...",job_1
2,https://www.indeed.com/viewjob?jk=6f84fbc9363b...,indeed,Front End Developer,EvenUp,https://www.indeed.com/cmp/Evenup,"San Francisco, CA, USA",fulltime,2024-11-11,True,EvenUp is on a mission to support injury law f...,job_2
3,https://www.indeed.com/viewjob?jk=b629c9dd1a94...,indeed,Solutions Engineer,"Riverbed Technology, Inc.",https://www.indeed.com/cmp/Riverbed-Technology,"North Chicago, IL, USA",fulltime,2024-11-11,True,"Riverbed. Empower the Experience:\n\nRiverbed,...",job_3
4,https://www.indeed.com/viewjob?jk=e634d4138294...,indeed,Senior Data Scientist,Roku,https://www.indeed.com/cmp/Roku-1,"New York, NY, USA",fulltime,2024-11-11,False,## **Teamwork makes the stream work.**\n\n###...,job_4


In [1]:
from langchain_text_splitters import CharacterTextSplitter

text_splitter = CharacterTextSplitter(
    # 텍스트를 분할할 때 사용할 구분자를 지정합니다. 기본값은 "\n\n"입니다.
    # separator=" ",
    # 분할된 텍스트 청크의 최대 크기를 지정합니다.
    chunk_size=4000,
    # 분할된 텍스트 청크 간의 중복되는 문자 수를 지정합니다.
    chunk_overlap=0,
    # 텍스트의 길이를 계산하는 함수를 지정합니다.
    length_function=len,
    # 구분자가 정규식인지 여부를 지정합니다.
    is_separator_regex=False,
)


In [37]:
total_chunks = []
for i, desciption in enumerate(JD_csv['description']):
    chunk = text_splitter.create_documents([desciption])
    chunk[0].metadata = JD_csv.iloc[i].to_dict()
    total_chunks.extend(chunk)

In [38]:
print(total_chunks)

[Document(metadata={'job_url': 'https://www.indeed.com/viewjob?jk=301b28f66c774252', 'site': 'indeed', 'title': 'Engineer', 'company': 'Realign LLC', 'company_url': 'https://www.indeed.com/cmp/Realign', 'location': 'Minneapolis, MN, USA', 'job_type': 'contract', 'date_posted': '2024-11-12', 'is_remote': False, 'description': "#####  **Job Type: Contract**\n\n##### **Job Category: IT**\n\n#### **Job Description**\n\nJob Title: Azure GenAI Engineer\n\nJob Summary:\n\nWe are seeking a highly skilled Azure GenAI Engineer to join our team and help\ndrive the development and implementation of cutting-edge artificial\nintelligence solutions on the Azure platform. The ideal candidate will have a\ndeep understanding of Azure services, machine learning models, and data\nanalytics to create robust and scalable AI solutions for our clients.\n\nResponsibilities and Duties:\n\n  * Design, develop, and deploy artificial intelligence models on the Azure platform\n  * Collaborate with cross-functional 

In [39]:
# embedding
from langchain_openai import ChatOpenAI, OpenAIEmbeddings

embedding = OpenAIEmbeddings()

In [ ]:
# 한번만 실행
# vector store
from langchain_chroma import Chroma

DB_PATH = "./chroma_db"

# 청크를 디스크에 저장. 저장시 persist_directory에 저장할 경로 지정
# VectorDB에 저장할 때 임베딩 모델도 지정 (이 경우는 OpenAIEmbeddings)
db = Chroma.from_documents(
    total_chunks, OpenAIEmbeddings(), persist_directory=DB_PATH, collection_name="chromaDB_0"
)


In [9]:
from langchain_chroma import Chroma
from langchain_openai import ChatOpenAI, OpenAIEmbeddings

embedding = OpenAIEmbeddings()
DB_PATH = "./chroma_db"

# 디스크에서 문서를 로드
persist_db = Chroma(
    persist_directory=DB_PATH,
    embedding_function=OpenAIEmbeddings(),
    collection_name="chromaDB_0",
)

In [10]:
persist_db.get()

{'ids': ['d74f8b57-3d6a-4692-bf1f-613f57db5991',
  'ba67e26a-9890-4b72-851f-4b4de5c56774',
  '010bd889-9955-428c-ab0b-96e70dc0afad',
  'a6369b0b-a07f-4aeb-b98b-95685a645dae',
  '54810c6a-8dfa-466d-bffe-cb20e89c7db3',
  '352b3db8-ba61-457b-a37b-dfe7a0e149ba',
  'd6489b76-8ad2-4e1c-adeb-376f3873e12c',
  'aaad4c1b-59e3-4358-9c23-615cb79866e1',
  '822fb208-6a93-4c9f-b999-f800eaf7fbf7',
  '5f1a9bfa-3e47-4859-aba3-79d9d978af0d',
  'f2617166-8a68-4bb7-b0ef-bc07dd594812',
  '5577ab1f-97b7-4353-b980-853d31b9eb4d',
  '045b5649-5537-425e-99d3-120e2dff2f19',
  '9d045b0e-ea35-4b47-a964-7bd2417616f4',
  'd162e60d-d348-4c1e-9acc-4075df6a935e',
  'b7502113-61cd-47f9-9426-3d9e85646883',
  '5f794a41-d0b4-4add-bb35-165bbc06ada4',
  'e1b32301-e21c-4046-9f50-a5addb6da533',
  '198a4430-220a-4037-a740-d5dfb3296969',
  'b1095dcc-a36a-4c2d-abd0-fecdba23e62c',
  'aed0ce2f-0c18-419b-b3b5-1b448e531429',
  '3963a4cb-f713-4b2e-b565-c7684a95c246',
  'b5ab91fc-df29-4dbc-a7c9-c40eb9d524d4',
  '448a6551-dbbb-4506-a144-

In [11]:
retriever = persist_db.as_retriever()

In [12]:
from langchain_core.prompts import PromptTemplate

prompt = PromptTemplate.from_template("""Based on the following two input data, evaluate how appealing the user’s resume would be to the company posting this JD. \n
The first text is the user’s resume, and the second text is the Job Description (JD).\n
Identify the key competencies required in the JD and assess whether these are included in the resume.\n

Then, rate the resume on a scale of 10 according to the criteria below:\n
Core Competency Match (1-5 points): Extract and evaluate the primary keywords and core competencies mentioned in the JD to see if they are included in the resume.\n
Additional Skills Match (1-3 points): Assess whether additional competencies or preferred skills from the JD are present in the resume.\n
Overall Assessment (1-2 points): Provide an overall assessment of how well the resume aligns with the JD and how appealing it may be for this position.\n

Output Requirements:\n
Rate the Core Competency and Additional Skills match on a scale of 10 based on the alignment between the JD and resume.\n
Explain in detail the reasoning behind this score.\n

Input data:
JD:\n
{context}

Resume:\n
{question}

Output format:
Score: 
Reason:
""")

In [ ]:
cv_path = './data/CV/ml_engineer_CV_0.txt'
with open (cv_path, "r") as file:
    cv = file.read()


In [14]:
retriever.invoke(cv)

[Document(metadata={'company': 'Amazon.com Services LLC', 'company_url': 'https://www.indeed.com/cmp/Amazon.com', 'date_posted': '2024-11-01', 'description': "* Master's degree in computer science, mathematics, statistics, machine learning or equivalent quantitative field\n  * Experience building machine learning models or developing algorithms for business application\n  * Experience in state-of-the-art deep learning models architecture design and deep learning training and optimization and model pruning\n  * Experience programming in Java, C++, Python or related language\n  * Experience with SQL and an RDBMS (e.g., Oracle) or Data Warehouse\n\nAmazon.com Buyer Risk Prevention (BRP) mission is to make Amazon the safest\nand most trusted place worldwide to transact online. BRP safeguards every\nfinancial transaction across all Amazon sites. As such, BRP designs and builds\nthe software systems, risk models, and operational processes that minimize\nrisk and maximize trust in Amazon.com.

In [ ]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough

def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)

llm = ChatOpenAI(model="gpt-3.5-turbo")

rag_chain = (
    {"context": retriever | format_docs, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

rag_chain.invoke(cv)

'Score: 6\n\nReason: \n\nCore Competency Match (1-5 points): The resume matches some of the core competencies mentioned in the JD, such as backend development experience, Docker skills, and experience in writing clean and efficient code. However, there is no specific mention of FastAPI, which is a key requirement in the JD. \n\nAdditional Skills Match (1-3 points): The resume does show additional skills in machine learning engineering and software development, which could be beneficial for the role. However, there is no mention of Kubernetes experience, which is a preferred skill in the JD.\n\nOverall Assessment (1-2 points): The resume aligns well with some of the core competencies required for the position, but it lacks specific experience in FastAPI and Kubernetes, which are mentioned in the JD. The resume showcases relevant experience and skills, but it could be more appealing if it included a direct match to all the key requirements mentioned in the JD.'